In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

In [3]:
#Função que transforma Linear para dB
def lin2db(x):
    return 10 * np.log10(x)

#Função que transforma dB para Linear
def db2lin(x):
    return 10 ** (x / 10)

In [2]:
# Função que gera as coordenadas dos APs
def distribuir_APs(num_aps):
    '''Distributes Access Points (APs) evenly within a square area.
    
    Parameters:
    num_aps (int): The number of APs to distribute. Must be a perfect square.
    
    Returns:
    np.array: An array of coordinates for the APs, or None if num_aps is not a perfect square.'''
    
    if num_aps not in [1, 4, 9, 16, 25, 36, 49, 64, 100]:
        return None

    tamanho_quadrado = 400
    lado_quadrado = int(np.sqrt(num_aps))

    tamanho_celula = tamanho_quadrado // lado_quadrado

    # Criar coordenadas usando meshgrid
    x, y = np.meshgrid(np.arange(0.5 * tamanho_celula, tamanho_quadrado, tamanho_celula),
                      np.arange(0.5 * tamanho_celula, tamanho_quadrado, tamanho_celula))

    coordenadas_APs = np.column_stack((x.ravel(), y.ravel()))

    return coordenadas_APs

In [ ]:
# Função que gera a distância entre a UE e a AP
def dAPUE(x_coord, y_coord, ap_coord):
  '''Calculate the Euclidean distance between a user equipment (UE) and an access point (AP).
    
    Parameters:
    ue_coords (tuple): A tuple (x_coord, y_coord) representing the coordinates of the UE.
    ap_coords (np.array): An array containing the coordinates of the APs.
    
    Returns:
    float: The Euclidean distance between the UE and the AP.
  '''
  dAPUE = np.linalg.norm(np.array([x_coord, y_coord]) - ap_coord)
  return dAPUE

In [ ]:
#Função que define o shadowing e muda ele a cada 10 passos
def find_shadowing(ue):
    '''Calculates the shadowing for a given user equipment (UE).

    Parameters:
    ue (int): The user equipment (UE) for which to calculate the shadowing.

    Returns:
    list: The shadowing value for the  each UE.
    '''
    shadowing = []
    for i in range(ue):
        shadowing.append(np.random.normal(0, 2))

    return shadowing 

In [ ]:
# Função que calcula o path gain
def find_path_gain(dist, shadowing):
    ''' Calculate the path gain for a given distance and shadowing value.

    Parameters:
    distance (float): The distance from the transmitter to the receiver.
    shadowing (float): The shadowing effect value, typically a multiplier derived from a log-normal distribution.

    Returns:
    float: The path gain calculated using the given distance and shadowing effects.'''
    beta = (30.5 + 36.7*np.log10(dist/1)) # dB

    path_gain = shadowing + beta # dB 

    path_gain_result = np.sqrt(db2lin(-path_gain)) # [LINEAR]

    return path_gain_result # [LINEAR]

In [ ]:
# Função que calcula a potência recebida
def find_pot_rec(pot_trans, dist, d_0, shadowing):
    ''''Calculate the received power from a transmitter to a receiver based on the distance, 
    transmission power, and shadowing effect.

    Parameters:
    pot_trans (float): The transmission power.
    dist (float): The distance from the transmitter to the receiver.
    d_0 (float): The threshold distance below which the power calculation does not occur.
    shadowing (float): The shadowing effect, modifying the power based on environmental factors.

    Returns:
    float: The received power if distance is above the threshold; otherwise, returns zero.'''
    
    beta = (30.5 + 36.7*np.log10(dist/1)) # dB

    if dist >= d_0:
        path_gain = shadowing + beta
        path_gain_result = np.sqrt(db2lin(-path_gain))
        pot_rec_result = path_gain_result * pot_trans # Obtem a potência recebida de cada AP
    else:
        pot_rec_result = 0

    return pot_rec_result # [LINEAR]

In [ ]:
def calculate_sinr()

In [ ]:
B_t, p_t, d_0, K_0 = 100e6, 1e-3, 1, 1e-17 # Em MHz, mW, metros, mW/Hz respectivamente
ap, ue, channel = 64, 13, 2
passos = 400000 # Definido já que a UE irá se mover a cada 10cm
velocity = 0.01 # Velocidade de 10 m/s considerando que cada iteração é equivalente a 10ms
shadow = find_shadowing(passos) # Definindo o shadowing que se altera para cada 10m
x_coord, y_coord = ue_moviment(velocity, passos) # Movimentando a UE
passos_array = np.arange(passos)